## 載入模型

In [94]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [95]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

In [96]:
# 載入模型與標準化器
base_model = load_model('DNN_best_model4mcAP.h5')

scaler = joblib.load('scaler_4mcAP.pkl')

In [97]:
# 凍結所有層
for layer in base_model.layers[:-1]:  # 除了最後一層 (Output Layer)
    layer.trainable = False

# 確認哪些層可訓練
base_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 64)                832       
                                                                 
 dense_45 (Dense)            (None, 128)               8320      
                                                                 
 dense_46 (Dense)            (None, 64)                8256      
                                                                 
 dense_47 (Dense)            (None, 49)                3185      
                                                                 
Total params: 20,593
Trainable params: 3,185
Non-trainable params: 17,408
_________________________________________________________________


In [98]:
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 加入新資料 & 資料前處理

In [99]:
selected_columns = ['Label',
                    'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
                    'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                    'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = ['AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
                              'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [ ]:
# 讀取測試資料
test_file_path = "timestamp_allignment_Balanced_2024_12_21_rtt_logs.csv"  # 測試資料的檔案名稱
date_test = "2024_12_21"
modelname = "DNN 4 mcAPs BEST"
test_data = pd.read_csv(test_file_path, usecols=selected_columns)
test_data

,Label,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,3999.0,7339.0,14219.0,-303.0,-65.0,-68.0,-65.0,-50.0,986.0,295.0,580.0,968.0
1,1-1,3921.0,7368.0,12835.0,-420.0,-65.0,-68.0,-64.0,-54.0,1071.0,1013.0,765.0,671.0
2,1-1,4038.0,6928.0,12510.0,-651.0,-65.0,-69.0,-64.0,-52.0,986.0,375.0,562.0,770.0
3,1-1,4038.0,7163.0,13955.0,-381.0,-65.0,-68.0,-65.0,-54.0,1042.0,267.0,527.0,555.0
4,1-1,4077.0,7163.0,13281.0,-342.0,-65.0,-68.0,-64.0,-54.0,996.0,137.0,826.0,428.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20673,9-11,1694.0,5083.0,2666.0,5438.0,-55.0,-67.0,-50.0,-66.0,294.0,1093.0,369.0,2384.0
20674,9-11,NaN,5229.0,NaN,NaN,NaN,-65.0,NaN,NaN,NaN,1125.0,NaN,NaN
20675,9-11,2007.0,5346.0,2627.0,4696.0,-55.0,-66.0,-50.0,-66.0,152.0,1073.0,482.0,463.0
20676,9-11,1929.0,5434.0,2549.0,4735.0,-55.0,-65.0,-51.0,-66.0,216.0,964.0,568.0,481.0


In [101]:
# 資料前處理 (一): 刪除前後n筆資料
n = 20
# 確保依據Label排序
test_data = test_data.sort_values(by=label_column).reset_index(drop=True)

# 建立一個空的 DataFrame 用於存放處理後的資料
test_processed_data = pd.DataFrame(columns=test_data.columns)

# 針對每個Label群組進行處理
for label, group in test_data.groupby(label_column):
    # 刪除前n筆和後n筆資料
    if len(group) > 2 * n:  # 確保群組資料足夠
        group = group.iloc[n:-n]
    else:
        group = pd.DataFrame()  # 若資料不足，刪除整個群組
    # 將處理後的群組資料加入
    test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)

# test_processed_data
# Calculate the number of rows with NaN values
nan_rows = test_processed_data.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_processed_data[test_processed_data.isnull().any(axis=1)]

# # 印出這些列
# print("Rows with NaN values:")
# print(rows_with_nan)
test_data_imputed = test_processed_data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()

# Calculate the number of rows with NaN values
nan_rows = test_data_imputed.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_data_imputed[test_data_imputed.isnull().any(axis=1)]

test_data_imputed

Number of rows with NaN values: 925
Number of rows with NaN values: 0


,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,3999.000000,7358.00000,13545.00000,-421.000000,-65.000000,-68.000000,-65.000000,-54.000000,993.000000,266.000000,610.000000,552.000000
1,1-1,1,3882.000000,7075.00000,13398.00000,-499.000000,-66.000000,-68.000000,-66.000000,-54.000000,978.000000,379.000000,738.000000,616.000000
2,1-1,2,4038.000000,7475.00000,13994.00000,-381.000000,-65.000000,-68.000000,-63.000000,-50.000000,949.000000,44.000000,589.000000,810.000000
3,1-1,3,3804.000000,7368.00000,14131.00000,-460.000000,-65.000000,-68.000000,-65.000000,-54.000000,1031.000000,93.000000,503.000000,603.000000
4,1-1,4,4077.000000,7309.00000,14043.00000,-381.000000,-65.000000,-68.000000,-65.000000,-54.000000,879.000000,125.000000,428.000000,541.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18713,9-11,18713,1807.000000,5112.00000,2510.00000,4863.076503,-45.000000,-65.000000,-50.000000,-65.371585,333.000000,1193.000000,566.000000,728.775956
18714,9-11,18714,1841.455526,5506.22043,2518.88587,4676.000000,-54.067385,-65.879032,-49.782609,-67.000000,252.994609,951.153226,598.269022,490.000000
18715,9-11,18715,1889.000000,5639.00000,2588.00000,4881.000000,-55.000000,-66.000000,-48.000000,-67.000000,221.000000,892.000000,564.000000,404.000000
18716,9-11,18716,2007.000000,5522.00000,2471.00000,4735.000000,-55.000000,-65.000000,-51.000000,-66.000000,202.000000,863.000000,605.000000,509.000000


In [102]:
reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

In [103]:
# 建立 Label 映射
y_test = test_data_imputed[target_column]
y_test_numeric = y_test.map(reverse_label_mapping)

print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
print("y_numeric unique values in DNN:", y_test_numeric.unique())

# 把label部分拿掉
X_test = test_data_imputed.drop(columns=['level_1','Label'])
# 確保測試資料的特徵與訓練資料的特徵一致
X_test = X_test[X_testing_selected_columns]  # 選取相同的特徵

X_test

Final reverse_label_mapping in DNN: {'1-1': 10, '1-2': 9, '1-3': 8, '1-4': 7, '1-5': 6, '1-6': 5, '1-7': 4, '1-8': 3, '1-9': 2, '1-10': 1, '1-11': 0, '2-1': 11, '2-11': 29, '3-1': 12, '3-11': 28, '4-1': 13, '4-11': 27, '5-1': 14, '5-11': 26, '6-1': 15, '6-2': 16, '6-3': 17, '6-4': 18, '6-5': 19, '6-6': 20, '6-7': 21, '6-8': 22, '6-9': 23, '6-10': 24, '6-11': 25, '7-1': 48, '7-11': 30, '8-1': 47, '8-11': 31, '9-1': 46, '9-11': 32, '10-1': 45, '10-11': 33, '11-1': 44, '11-2': 43, '11-3': 42, '11-4': 41, '11-5': 40, '11-6': 39, '11-7': 38, '11-8': 37, '11-9': 36, '11-10': 35, '11-11': 34}
y_numeric unique values in DNN: [10  1  0  9  8  7  6  5  4  3  2 45 33 44 35 34 43 42 41 40 39 38 37 36
 11 29 12 28 13 27 14 26 15 24 25 16 17 18 19 20 21 22 23 48 30 47 31 46
 32]


,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi
0,3999.000000,7358.00000,13545.00000,-421.000000,993.000000,266.000000,610.000000,552.000000,-65.000000,-68.000000,-65.000000,-54.000000
1,3882.000000,7075.00000,13398.00000,-499.000000,978.000000,379.000000,738.000000,616.000000,-66.000000,-68.000000,-66.000000,-54.000000
2,4038.000000,7475.00000,13994.00000,-381.000000,949.000000,44.000000,589.000000,810.000000,-65.000000,-68.000000,-63.000000,-50.000000
3,3804.000000,7368.00000,14131.00000,-460.000000,1031.000000,93.000000,503.000000,603.000000,-65.000000,-68.000000,-65.000000,-54.000000
4,4077.000000,7309.00000,14043.00000,-381.000000,879.000000,125.000000,428.000000,541.000000,-65.000000,-68.000000,-65.000000,-54.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
18713,1807.000000,5112.00000,2510.00000,4863.076503,333.000000,1193.000000,566.000000,728.775956,-45.000000,-65.000000,-50.000000,-65.371585
18714,1841.455526,5506.22043,2518.88587,4676.000000,252.994609,951.153226,598.269022,490.000000,-54.067385,-65.879032,-49.782609,-67.000000
18715,1889.000000,5639.00000,2588.00000,4881.000000,221.000000,892.000000,564.000000,404.000000,-55.000000,-66.000000,-48.000000,-67.000000
18716,2007.000000,5522.00000,2471.00000,4735.000000,202.000000,863.000000,605.000000,509.000000,-55.000000,-65.000000,-51.000000,-66.000000


In [104]:
# 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
X_test_scaled = scaler.transform(X_test)

print(len(X_test_scaled))

18718


### 重新訓練model

In [105]:
# 取出 10% 資料作為訓練資料，剩下的 90% 作為測試資料
X_train_small, X_unused, y_train_small, y_unused = train_test_split(
    X_test_scaled, y_test_numeric, test_size=0.9, random_state=42, stratify=y_test_numeric
)

In [106]:
base_model.fit(X_train_small, y_train_small, epochs=5, batch_size=32)

Epoch 1/5
59/59 [==============================] - 0s 850us/step - loss: 6.4279 - accuracy: 0.3597
Epoch 2/5
59/59 [==============================] - 0s 679us/step - loss: 4.1278 - accuracy: 0.4377
Epoch 3/5
59/59 [==============================] - 0s 601us/step - loss: 2.6994 - accuracy: 0.5168
Epoch 4/5
59/59 [==============================] - 0s 507us/step - loss: 1.7646 - accuracy: 0.6301
Epoch 5/5
59/59 [==============================] - 0s 334us/step - loss: 1.2048 - accuracy: 0.7253


In [107]:
# 使用剩餘的 90% 測試資料進行模型評估
loss, accuracy = base_model.evaluate(X_unused, y_unused)
print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

527/527 [==============================] - 0s 294us/step - loss: 1.1011 - accuracy: 0.7546
Evaluation on 90% unused data - Loss: 1.1011, Accuracy: 0.7546


In [108]:
# ### 預測
# base_model.evaluate(X_test_scaled, y_test_numeric)  # 確保模型在測試模式

In [109]:
# 預測測試資料
y_test_pred_numeric = base_model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

# 轉換為原本的 Label
y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
y_test_pred_labels

585/585 [==============================] - 0s 251us/step


['1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-5',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-5',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-6',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',


In [110]:
# 讀取測試資料的實際 Label
y_test_actual = test_data_imputed[target_column]
test_data_imputed

,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,3999.000000,7358.00000,13545.00000,-421.000000,-65.000000,-68.000000,-65.000000,-54.000000,993.000000,266.000000,610.000000,552.000000
1,1-1,1,3882.000000,7075.00000,13398.00000,-499.000000,-66.000000,-68.000000,-66.000000,-54.000000,978.000000,379.000000,738.000000,616.000000
2,1-1,2,4038.000000,7475.00000,13994.00000,-381.000000,-65.000000,-68.000000,-63.000000,-50.000000,949.000000,44.000000,589.000000,810.000000
3,1-1,3,3804.000000,7368.00000,14131.00000,-460.000000,-65.000000,-68.000000,-65.000000,-54.000000,1031.000000,93.000000,503.000000,603.000000
4,1-1,4,4077.000000,7309.00000,14043.00000,-381.000000,-65.000000,-68.000000,-65.000000,-54.000000,879.000000,125.000000,428.000000,541.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18713,9-11,18713,1807.000000,5112.00000,2510.00000,4863.076503,-45.000000,-65.000000,-50.000000,-65.371585,333.000000,1193.000000,566.000000,728.775956
18714,9-11,18714,1841.455526,5506.22043,2518.88587,4676.000000,-54.067385,-65.879032,-49.782609,-67.000000,252.994609,951.153226,598.269022,490.000000
18715,9-11,18715,1889.000000,5639.00000,2588.00000,4881.000000,-55.000000,-66.000000,-48.000000,-67.000000,221.000000,892.000000,564.000000,404.000000
18716,9-11,18716,2007.000000,5522.00000,2471.00000,4735.000000,-55.000000,-65.000000,-51.000000,-66.000000,202.000000,863.000000,605.000000,509.000000


In [111]:
# 取得預測與實際座標
y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

# 計算 MDE (Mean Distance Error)
distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
mean_mde = np.mean(distances)

# 記錄每個 RP 的 MDE
mde_report_test = {}
for true_label, distance in zip(y_test_actual, distances):
    if true_label not in mde_report_test:
        mde_report_test[true_label] = []
    mde_report_test[true_label].append(distance)

# 計算測試資料的 MDE 平均值
mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

# 儲存 MDE 結果到 JSON 檔案
test_file_path = f"{modelname}_{date_test}.json"
with open(test_file_path, "w") as f:
    json.dump(mde_report_test_avg, f, indent=4)

print(f"Test Data MDE report saved to: {test_file_path}")
print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")


Test Data MDE report saved to: DNN 4 mcAPs BEST_2025_01_10.json

Test Data Mean MDE: 0.3003 meters
